### Create SPARK_HOME and PYLIB env var and update PATH env var

In [1]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

### Initializing Spark

Build __SparkConf__ object 

    Contains information about your application.  


Create __SparkContext__ object 
    
    Tells Spark how to access a cluster. 
    

Create __SparkSession__ object

    The entry point to programming Spark with the Dataset and DataFrame API.

    Used to create DataFrame, register DataFrame as tables and execute SQL over tables etc.

In [2]:
from pyspark.conf import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

conf = SparkConf().setAppName("Universal Bank Data Set").setMaster('local')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

### Loading the dependent libraries

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
# from pyspark.sql.functions import isnan, when, count, col, countDistinct


#### Problem Statement
The dataset is from a bank, data related to direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, to access if the product (bank term deposit) would be (or not) subscribed. The data and attribute description are in the folder. 


#### Data Dictionary
 The dataset has the following attributes:

1 - age (numeric)

2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
                                    "blue-collar","self-employed","retired","technician","services") 

3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)

4 - education (categorical: "unknown","secondary","primary","tertiary")

5 - default: has credit in default? (binary: "yes","no")

#### 6 - balance: average yearly balance, in euros (numeric) 

7 - housing: has housing loan? (binary: "yes","no")

8 - loan: has personal loan? (binary: "yes","no")

9 - contact: contact communication type (categorical: "unknown","telephone","cellular") 

10 - day: last contact day of the month (numeric)

11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")

12 - duration: last contact duration, in seconds (numeric)

13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
  
15 - previous: number of contacts performed before this campaign and for this client (numeric)

16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

17 - Approved_no_yes - has the client subscribed to a __term deposit?__ (binary: "yes","no")

### Defining the schema to data

In [4]:
## Define Schema
bankDataSchema = StructType([
    StructField("age", IntegerType(), True),
    StructField("job", StringType(), True),
    StructField("marital_status", StringType(), True),
    StructField("education", StringType(), True),
    StructField("default", StringType(), True),
    StructField("balance", DoubleType(), True),
    StructField("housing", StringType(), True),
    StructField("loan", StringType(), True),        
    StructField("contact", StringType(), True),
    StructField("day", IntegerType(), True),
    StructField("month", StringType(), True),
    StructField("duration", DoubleType(), True),
    StructField("campaign", DoubleType(), True),
    StructField("pdays", DoubleType(), True),
    StructField("previous", DoubleType(), True),
    StructField("poutcome", StringType(), True),
    StructField("Approved_no_yes", StringType(), True)])

### Reading the data and creating a dataframe

In [5]:
## Read data and create a dataframe
data = spark.read.format("csv")\
       .option("header", "false")\
       .option("inferSchema", "true")\
       .load("hdfs:///user/jeevans/bankdata/", schema = bankDataSchema)

### Understanding Data

#### Print Schema

In [6]:
data.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: double (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- campaign: double (nullable = true)
 |-- pdays: double (nullable = true)
 |-- previous: double (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- Approved_no_yes: string (nullable = true)



#### Another way to check the data type of each attribute

In [7]:
data.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital_status', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('balance', 'double'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('day', 'int'),
 ('month', 'string'),
 ('duration', 'double'),
 ('campaign', 'double'),
 ('pdays', 'double'),
 ('previous', 'double'),
 ('poutcome', 'string'),
 ('Approved_no_yes', 'string')]

#### Total number of Columns and Records

In [8]:
print("No. of Columns = {}".format(len(data.columns)))

print('No. of Records = {}'.format(data.count()))

No. of Columns = 17
No. of Records = 4521


#### Look at first 3 row of the dataframe

In [9]:
data.head(3)

[Row(age=30, job=u'unemployed', marital_status=u'married', education=u'primary', default=u'no', balance=1787.0, housing=u'no', loan=u'no', contact=u'cellular', day=19, month=u'oct', duration=79.0, campaign=1.0, pdays=-1.0, previous=0.0, poutcome=u'unknown', Approved_no_yes=u'no'),
 Row(age=33, job=u'services', marital_status=u'married', education=u'secondary', default=u'no', balance=4789.0, housing=u'yes', loan=u'yes', contact=u'cellular', day=11, month=u'may', duration=220.0, campaign=1.0, pdays=339.0, previous=4.0, poutcome=u'failure', Approved_no_yes=u'no'),
 Row(age=35, job=u'management', marital_status=u'single', education=u'tertiary', default=u'no', balance=1350.0, housing=u'yes', loan=u'no', contact=u'cellular', day=16, month=u'apr', duration=185.0, campaign=1.0, pdays=330.0, previous=1.0, poutcome=u'failure', Approved_no_yes=u'no')]

In [10]:
data.show(3)

+---+----------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---------------+
|age|       job|marital_status|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|
+---+----------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---------------+
| 30|unemployed|       married|  primary|     no| 1787.0|     no|  no|cellular| 19|  oct|    79.0|     1.0| -1.0|     0.0| unknown|             no|
| 33|  services|       married|secondary|     no| 4789.0|    yes| yes|cellular| 11|  may|   220.0|     1.0|339.0|     4.0| failure|             no|
| 35|management|        single| tertiary|     no| 1350.0|    yes|  no|cellular| 16|  apr|   185.0|     1.0|330.0|     1.0| failure|             no|
+---+----------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----

#### Summary statistics

In [11]:
data.describe().show()

+-------+------------------+-------+--------------+---------+-------+------------------+-------+----+--------+------------------+-----+------------------+------------------+------------------+------------------+--------+---------------+
|summary|               age|    job|marital_status|education|default|           balance|housing|loan| contact|               day|month|          duration|          campaign|             pdays|          previous|poutcome|Approved_no_yes|
+-------+------------------+-------+--------------+---------+-------+------------------+-------+----+--------+------------------+-----+------------------+------------------+------------------+------------------+--------+---------------+
|  count|              4521|   4521|          4521|     4521|   4521|              4521|   4521|4521|    4521|              4521| 4521|              4521|              4521|              4521|              4521|    4521|           4521|
|   mean| 41.17009511170095|   null|          null| 

Show only fixed set of colums 

In [12]:
data.describe().select('summary', 'age', 'loan', 'balance', 'pdays').show()

+-------+------------------+----+------------------+------------------+
|summary|               age|loan|           balance|             pdays|
+-------+------------------+----+------------------+------------------+
|  count|              4521|4521|              4521|              4521|
|   mean| 41.17009511170095|null|1422.6578190665782|39.766644547666445|
| stddev|10.576210958711263|null|3009.6381424673395|100.12112444301656|
|    min|                19|  no|           -3313.0|              -1.0|
|    max|                87| yes|           71188.0|             871.0|
+-------+------------------+----+------------------+------------------+



Observation

    Balance has -ve values

In [13]:
data.where(data.balance < 0).count()

366

### Data Preprocessing

#### Replace negative balances with zeroes

In [14]:
data = data.withColumn('balance', when(data.balance > 0, data.balance).otherwise(0))

#### Handling missing values

Checking for null values at each column

In [15]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+---+---+--------------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---------------+
|age|job|marital_status|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|
+---+---+--------------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---------------+
|  0|  0|             0|        0|      0|      0|      0|   0|      0|  0|    0|       0|       0|    0|       0|       0|              0|
+---+---+--------------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---------------+



#### Split the data into training and test sets (30% held out for testing)

In [16]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

#### Creating a list of categorical and numerical features

In [17]:
cat_Var_Names = ['job', 'marital_status', 'education', 'default', 'housing', 
                 'day', 'contact', 'month', 'poutcome', 'Approved_no_yes']

num_Var_Names = ['age', 'balance', 'duration', 'previous', 'pdays', 'campaign']

#### Use VectorAssembler to combine a given list of numcolumns into a single vector column.

In [18]:
from pyspark.ml.feature import VectorAssembler

assembler_Num = VectorAssembler(inputCols=num_Var_Names, outputCol="num_features")

#### Scale all the numeric attributes using MinMaxScaler

    MinMaxScaler transforms a dataset of Vector rows, rescaling each feature to a specific range (often [0, 1]). 

    MinMaxScaler computes summary statistics on a data set and produces a MinMaxScalerModel. The model can then transform each feature individually such that it is in the given range.

In [19]:
from pyspark.ml.feature import MinMaxScaler

min_Max_Scalar = MinMaxScaler(inputCol="num_features", outputCol="scaled_num_features")

#### Covert categorical to numeric : 

    OneHotEncoder, StringIndexer, VectorAssembler,  VectorIndexer

In [20]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

indexers_Cat = [StringIndexer(inputCol=cat_Var_Name, outputCol="{0}_index".format(cat_Var_Name)) for cat_Var_Name in cat_Var_Names ]
encoders_Cat = [OneHotEncoder(inputCol=indexer.getOutputCol(), outputCol="{0}_vec".format(indexer.getInputCol())) for indexer in indexers_Cat]
assembler_Cat = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders_Cat], outputCol="cat_features")

assembler = VectorAssembler(inputCols=["scaled_num_features","cat_features"], outputCol="features")


In [21]:
indexer_Label = StringIndexer(inputCol="loan", outputCol="label")

In [22]:
preprocessiong_Stages = [assembler_Num]+[min_Max_Scalar]+indexers_Cat+encoders_Cat+[assembler_Cat]+[assembler]+[indexer_Label]

### Model Building and Evaluation

#### Logistic Regression

In [23]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=10, labelCol="label", featuresCol="features")

In [24]:
from pyspark.ml import Pipeline

lr_Pipeline = Pipeline(stages=preprocessiong_Stages+[lr]) 

lr_Pipeline_model = lr_Pipeline.fit(trainingData)

In [25]:
print("Coefficients: " + str(lr_Pipeline_model.stages[-1].coefficients))
print("Intercept: " + str(lr_Pipeline_model.stages[-1].intercept))

Coefficients: [-0.710121916343,-4.22902246315,0.354450780907,0.96947748232,0.27027464059,0.594143226419,0.125406707623,-0.00525985643844,0.107542040447,0.218256974689,0.11076213061,-0.000603590384907,0.666917451831,-0.094787492947,-0.548774363689,0.129175403917,-1.56240534608,0.124815247247,-0.226883832814,0.40103725583,0.149467438393,-0.0972437476018,-1.14962987804,-0.0544964913157,-0.407736749923,-0.0858331192117,-0.403963999896,0.219865778816,0.387569612049,0.628272118039,0.318435467906,0.305670414001,0.161814851413,-0.181409055233,-0.230825819583,0.241599806793,0.258564743629,0.0947740436397,-0.196137456599,-0.213754811455,0.486203492319,-0.291596052914,-0.67092955477,-0.0418712904755,0.100112210365,0.159000451578,0.15550132615,-0.425501251642,0.0687629051454,-1.25397577177,0.874864751866,0.165986768255,-0.984977941963,-0.228463529061,-0.061212600242,0.13486363404,-0.253825360017,0.970952231844,-0.257997951761,-0.239287136223,0.502444321536,-0.21306340475,-0.0402448583029,-0.395961

In [26]:
lr_Summary = lr_Pipeline_model.stages[-1].summary
objectiveHistory = lr_Summary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

objectiveHistory:
0.415645100646
0.394291354729
0.388490505959
0.388126580716
0.387368409818
0.385654601826
0.383891342298
0.382288921054
0.381840536009
0.381659163401
0.381388955797


In [27]:
train_predictions_lr = lr_Pipeline_model.transform(trainingData)
test_predictions_lr = lr_Pipeline_model.transform(testData)

In [28]:
test_predictions_lr.show(2)

+---+-------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---------------+--------------------+--------------------+---------+--------------------+---------------+-------------+-------------+---------+-------------+-----------+--------------+---------------------+---------------+------------------+-------------+-------------+-----------+---------------+-------------+--------------+-------------+-------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|age|    job|marital_status|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|        num_features| scaled_num_features|job_index|marital_status_index|education_index|default_index|housing_index|day_index|contact_index|month_index|poutcome_index|Approved_no_yes_index|        job_vec|marital_status_vec|education_vec|  default_vec|housing_vec| 

#### Evaluation : LR Model

In [29]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

predictionAndLabels_train_lr = train_predictions_lr.select("prediction", "label")
train_accuracy_lr = evaluator.evaluate(predictionAndLabels_train_lr)

print("Train accuracy  = " + str(train_accuracy_lr))

predictionAndLabels_test_lr = test_predictions_lr.select("prediction", "label")
test_accuracy_lr = evaluator.evaluate(predictionAndLabels_test_lr)

print("Test accuracy = " + str(test_accuracy_lr))

Train accuracy  = 0.855611601513
Test accuracy = 0.833209785026


#### Tuning LR Model

In [30]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1]) \
    .addGrid(lr.elasticNetParam, [0.5])\
    .build()
    
lr_crossval = CrossValidator(estimator=lr_Pipeline,
                             estimatorParamMaps=paramGrid,
                             evaluator=evaluator,
                             numFolds=2)     

In [31]:
# Run cross-validation, and choose the best set of parameters.
lr_crossval_Model = lr_crossval.fit(trainingData)

In [32]:
train_predictions_lrcv = lr_crossval_Model.transform(trainingData)
test_predictions_lrcv = lr_crossval_Model.transform(testData)

In [33]:
predictionAndLabels_train_lrcv = train_predictions_lrcv.select("prediction", "label")
train_accuracycv = evaluator.evaluate(predictionAndLabels_train_lrcv)
print("Train set accuracy  = " + str(train_accuracycv))

predictionAndLabels_test_lrcv = test_predictions_lrcv.select("prediction", "label")
test_accuracycv = evaluator.evaluate(predictionAndLabels_test_lrcv)
print("Test set accuracy = " + str(test_accuracycv))

Train set accuracy  = 0.854035308953
Test set accuracy = 0.830985915493


#### Decision Tree Classifier

In [34]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [35]:
dt_Pipeline = Pipeline(stages=preprocessiong_Stages+[dt]) 

dt_Pipeline_model = dt_Pipeline.fit(trainingData)

In [36]:
train_predictions_dt = dt_Pipeline_model.transform(trainingData)
test_predictions_dt = dt_Pipeline_model.transform(testData)

In [37]:
test_predictions_dt.show(2)

+---+-------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---------------+--------------------+--------------------+---------+--------------------+---------------+-------------+-------------+---------+-------------+-----------+--------------+---------------------+---------------+------------------+-------------+-------------+-----------+---------------+-------------+--------------+-------------+-------------------+--------------------+--------------------+-----+-------------+--------------------+----------+
|age|    job|marital_status|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|        num_features| scaled_num_features|job_index|marital_status_index|education_index|default_index|housing_index|day_index|contact_index|month_index|poutcome_index|Approved_no_yes_index|        job_vec|marital_status_vec|education_vec|  default_vec|housing_vec|        

#### Evaluation : DT Model

In [38]:
predictionAndLabels_train_dt = train_predictions_dt.select("prediction", "label")
train_accuracy_dt = evaluator.evaluate(predictionAndLabels_train_dt)

print("Train accuracy  = " + str(train_accuracy_dt))

predictionAndLabels_test_dt = test_predictions_dt.select("prediction", "label")
test_accuracy_dt = evaluator.evaluate(predictionAndLabels_test_dt)

print("Test accuracy = " + str(test_accuracy_dt))

Train accuracy  = 0.860340479193
Test accuracy = 0.82357301705


#### Random Forest Classifier

In [39]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

In [40]:
rf_Pipeline = Pipeline(stages=preprocessiong_Stages+[rf]) 

rf_Pipeline_model = rf_Pipeline.fit(trainingData)

In [41]:
train_predictions_rf = rf_Pipeline_model.transform(trainingData)
test_predictions_rf = rf_Pipeline_model.transform(testData)

In [42]:
test_predictions_rf.show(2)

+---+-------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---------------+--------------------+--------------------+---------+--------------------+---------------+-------------+-------------+---------+-------------+-----------+--------------+---------------------+---------------+------------------+-------------+-------------+-----------+---------------+-------------+--------------+-------------+-------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|age|    job|marital_status|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|        num_features| scaled_num_features|job_index|marital_status_index|education_index|default_index|housing_index|day_index|contact_index|month_index|poutcome_index|Approved_no_yes_index|        job_vec|marital_status_vec|education_vec|  default_vec|housing_vec| 

#### Evaluation : RF Model

In [43]:
predictionAndLabels_train_rf = train_predictions_rf.select("prediction", "label")
train_accuracy_rf = evaluator.evaluate(predictionAndLabels_train_rf)

print("Train accuracy  = " + str(train_accuracy_rf))

predictionAndLabels_test_rf = test_predictions_rf.select("prediction", "label")
test_accuracy_rf = evaluator.evaluate(predictionAndLabels_test_rf)

print("Test accuracy = " + str(test_accuracy_rf))

Train accuracy  = 0.854035308953
Test accuracy = 0.830985915493


#### Gradient Boosted Tree Classifier

In [44]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)

In [45]:
gbt_Pipeline = Pipeline(stages=preprocessiong_Stages+[gbt]) 

gbt_Pipeline_model = gbt_Pipeline.fit(trainingData)

In [46]:
train_predictions_gbt = gbt_Pipeline_model.transform(trainingData)
test_predictions_gbt = gbt_Pipeline_model.transform(testData)

In [47]:
test_predictions_gbt.show(2)

+---+-------+--------------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---------------+--------------------+--------------------+---------+--------------------+---------------+-------------+-------------+---------+-------------+-----------+--------------+---------------------+---------------+------------------+-------------+-------------+-----------+---------------+-------------+--------------+-------------+-------------------+--------------------+--------------------+-----+----------+
|age|    job|marital_status|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|Approved_no_yes|        num_features| scaled_num_features|job_index|marital_status_index|education_index|default_index|housing_index|day_index|contact_index|month_index|poutcome_index|Approved_no_yes_index|        job_vec|marital_status_vec|education_vec|  default_vec|housing_vec|        day_vec|  contact_vec|     month_ve

#### Evaluation : gbt Model

In [48]:
predictionAndLabels_train_gbt = train_predictions_gbt.select("prediction", "label")
train_accuracy_gbt = evaluator.evaluate(predictionAndLabels_train_gbt)

print("Train accuracy  = " + str(train_accuracy_gbt))

predictionAndLabels_test_gbt = test_predictions_gbt.select("prediction", "label")
test_accuracy_gbt = evaluator.evaluate(predictionAndLabels_test_gbt)

print("Test accuracy = " + str(test_accuracy_gbt))

Train accuracy  = 0.866015132409
Test accuracy = 0.824314306894
